In [ ]:
import requests
import json
import pickle



def get_bne_wikidata(max_number=100):
    key = "bne"
    value = "wikidata"
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
    SELECT ?bne ?wikidata
    WHERE { 
        ?wikidata wdt:P950 ?bne .
    }
    LIMIT """+str(max_number)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    json_data = response.json()['results']['bindings']
    return [(item[key]['value'], item[value]['value']) for item in json_data]


datos = get_bne_wikidata()
#for key,value in datos:
    #print(key,"\t",value)

f = open("wikidatamap.bin", "wb+")
pickle.dump(datos, f)

In [ ]:
import importlib
import pickle
import dataset
importlib.reload(dataset)


datos = dataset.Dataset()
query2 = """PREFIX wikibase: <http://wikiba.se/ontology>
construct { ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }
WHERE { ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
    ?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
} LIMIT 10000 OFFSET 1880000
"""
datos.load_dataset_from_query(query2, only_uri=False)


#datos.load_dataset_from_nlevels(5, extra_params="LIMIT 100")
datos.show(verbose=False)
dataset1 = datos.train_split()

#     print(sub,dataset1[sub])
datos.save_to_binary("wikidata_2.bin")

# datos.subs

#print(len(jsonlist))
#jsonlist

In [13]:
import importlib
import pickle
import kgeserver.dataset as dataset
importlib.reload(dataset)


datos = dataset.Dataset()
fb_test = open("FB15k/freebase_mtr100_mte100-test.txt", "r")
fb_train = open("FB15k/freebase_mtr100_mte100-train.txt", "r")
fb_valid = open("FB15k/freebase_mtr100_mte100-valid.txt", "r")
datos.load_dataset_from_csv(fb_train, "\t")
datos.load_dataset_from_csv(fb_test, "\t")
datos.load_dataset_from_csv(fb_valid, "\t")

datos.show(verbose=False)



14951 entities, 1345 relations, 592213 tripletas


In [ ]:
import importlib
import kgeserver.dataset as dataset
import kgeserver.wikidata_dataset as wikidata_dataset
import pickle
importlib.reload(dataset)
importlib.reload(wikidata_dataset)
from datetime import datetime

dtset = wikidata_dataset.WikidataDataset()

sv = dtset.get_seed_vector(verbose=2)
#dataset.load_entire_dataset(1)
dtset.load_dataset_recurrently(2, sv, verbose=2)
dtset.show()



In [22]:
print(dtset.relations[8])
dtset.save_to_binary("wikidata_25k.bin")

P31


True

In [ ]:
import importlib
import dataset
import algorithm
import skge
importlib.reload(dataset)
importlib.reload(algorithm)


dt = dataset.Dataset()

dt.load_from_binary("wn18.bin")

al = algorithm.Algorithm(dt)

models = models = al.find_best(ncomps=[50], margins=[2], model_types=[skge.TransE])

In [1]:
dic = {"a":1234, "b":5678}
l = [1234, 5678]
s = 2
%timeit dic["b"]=4567
%timeit l[1]=4567
%timeit s = 4567

print(dic, l, s)

The slowest run took 28.27 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 66.3 ns per loop
10000000 loops, best of 3: 54.5 ns per loop
100000000 loops, best of 3: 15.9 ns per loop
{'a': 1234, 'b': 4567} [1234, 4567] 2


In [7]:
wikidataQueries = {
    "where": ("{0} ?predicate ?object . "
              "?predicate a owl:ObjectProperty . "
              "FILTER NOT EXISTS {{ ?object a wikibase:BestRank }}")
}
wikidata_id=50
el_query = """SELECT ({0} as ?subject) ?predicate ?object
    WHERE {{
      {1}
    }}""".format("wd:Q{0}".format(wikidata_id),wikidataQueries["where"].format("wd:Q{0}".format(wikidata_id)))

print(el_query)

SELECT (wd:Q50 as ?subject) ?predicate ?object
    WHERE {
      wd:Q50 ?predicate ?object . ?predicate a owl:ObjectProperty . FILTER NOT EXISTS { ?object a wikibase:BestRank }
    }
